In [48]:
import json
import requests
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_absolute_error, accuracy_score

In [2]:
response = requests.get('https://min-api.cryptocompare.com/data/histoday?fsym=ETH&tsym=CAD&limit=1000')
json.loads(response.content)['Data']

[{'time': 1544832000,
  'high': 117.63,
  'low': 116.46,
  'open': 117.5,
  'volumefrom': 2080434.83,
  'volumeto': 244071934.11,
  'close': 117.32,
  'conversionType': 'multiply',
  'conversionSymbol': 'BTC'},
 {'time': 1544918400,
  'high': 123.85,
  'low': 121.02,
  'open': 117.32,
  'volumefrom': 2147007.94,
  'volumeto': 261332793.28,
  'close': 121.72,
  'conversionType': 'multiply',
  'conversionSymbol': 'BTC'},
 {'time': 1545004800,
  'high': 135.13,
  'low': 130.55,
  'open': 121.72,
  'volumefrom': 2953534.78,
  'volumeto': 394553837.36,
  'close': 133.59,
  'conversionType': 'multiply',
  'conversionSymbol': 'BTC'},
 {'time': 1545091200,
  'high': 144.52,
  'low': 138.9,
  'open': 133.59,
  'volumefrom': 3183833.06,
  'volumeto': 458970055.08,
  'close': 144.16,
  'conversionType': 'multiply',
  'conversionSymbol': 'BTC'},
 {'time': 1545177600,
  'high': 158.69,
  'low': 151.81,
  'open': 144.16,
  'volumefrom': 3110771.48,
  'volumeto': 472777535.66,
  'close': 151.98,
  'c

In [3]:
df = pd.DataFrame(json.loads(response.content)['Data'])
df.head()

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,1544832000,117.63,116.46,117.50,2080434.83,2.440719e+08,117.32,multiply,BTC
1,1544918400,123.85,121.02,117.32,2147007.94,2.613328e+08,121.72,multiply,BTC
2,1545004800,135.13,130.55,121.72,2953534.78,3.945538e+08,133.59,multiply,BTC
3,1545091200,144.52,138.90,133.59,3183833.06,4.589701e+08,144.16,multiply,BTC
4,1545177600,158.69,151.81,144.16,3110771.48,4.727775e+08,151.98,multiply,BTC


In [4]:
df = df.set_index('time')

In [5]:
df.head()

,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
time,,,,,,,,
1544832000,117.63,116.46,117.50,2080434.83,2.440719e+08,117.32,multiply,BTC
1544918400,123.85,121.02,117.32,2147007.94,2.613328e+08,121.72,multiply,BTC
1545004800,135.13,130.55,121.72,2953534.78,3.945538e+08,133.59,multiply,BTC
1545091200,144.52,138.90,133.59,3183833.06,4.589701e+08,144.16,multiply,BTC
1545177600,158.69,151.81,144.16,3110771.48,4.727775e+08,151.98,multiply,BTC


In [6]:
df = df[['high', 'low', 'open', 'volumefrom', 'volumeto', 'close']]
df.head()

,high,low,open,volumefrom,volumeto,close
time,,,,,,
1544832000,117.63,116.46,117.50,2080434.83,2.440719e+08,117.32
1544918400,123.85,121.02,117.32,2147007.94,2.613328e+08,121.72
1545004800,135.13,130.55,121.72,2953534.78,3.945538e+08,133.59
1545091200,144.52,138.90,133.59,3183833.06,4.589701e+08,144.16
1545177600,158.69,151.81,144.16,3110771.48,4.727775e+08,151.98


In [7]:
def train_test_split(df):
    split_row = len(df) - int(0.1 * len(df))
    train_data = df.iloc[:split_row]
    test_data = df.iloc[split_row:]
    return train_data, test_data
train, test = train_test_split(df)

In [8]:
len(test)

100

In [9]:
def extract_windows_data(df):
    window_data = []
    for idx in range(len(df) - 10):
        tmp = df[idx: (idx + 10)].copy()
        window_data.append(tmp.values)
    return np.array(window_data)

In [31]:
def prepare_data(df):
    train_data, test_data = train_test_split(df)
    X_train = extract_windows_data(train_data)
    X_test = extract_windows_data(test_data)
    y_train = train_data['close'][10:].values
    y_test = test_data['close'][10:].values
    
    y_train = train_data['close'][:-10].values
    y_test = test_data['close'][:-10].values
    return train_data, test_data, X_train, X_test, y_train, y_test

In [32]:
df.iloc[:-1].reset_index(drop=True)

,high,low,open,volumefrom,volumeto,close
0,117.63,116.46,117.50,2080434.83,2.440719e+08,117.32
1,123.85,121.02,117.32,2147007.94,2.613328e+08,121.72
2,135.13,130.55,121.72,2953534.78,3.945538e+08,133.59
3,144.52,138.90,133.59,3183833.06,4.589701e+08,144.16
4,158.69,151.81,144.16,3110771.48,4.727775e+08,151.98
...,...,...,...,...,...,...
995,5116.89,4942.69,4878.13,255535.08,1.267681e+09,4960.89
996,5049.58,4897.66,4960.89,250990.26,1.230751e+09,4903.58
997,4462.18,4167.23,4903.58,565756.87,2.459038e+09,4346.46
998,4449.86,4210.29,4346.46,448913.38,1.986875e+09,4425.96


In [33]:
df.iloc[1:].reset_index(drop=True)

,high,low,open,volumefrom,volumeto,close
0,123.85,121.02,117.32,2147007.94,2.613328e+08,121.72
1,135.13,130.55,121.72,2953534.78,3.945538e+08,133.59
2,144.52,138.90,133.59,3183833.06,4.589701e+08,144.16
3,158.69,151.81,144.16,3110771.48,4.727775e+08,151.98
4,171.71,160.66,151.98,3233173.04,5.458350e+08,168.82
...,...,...,...,...,...,...
995,5049.58,4897.66,4960.89,250990.26,1.230751e+09,4903.58
996,4462.18,4167.23,4903.58,565756.87,2.459038e+09,4346.46
997,4449.86,4210.29,4346.46,448913.38,1.986875e+09,4425.96
998,4488.34,4317.34,4425.96,282681.29,1.222251e+09,4323.78


In [34]:
df.iloc[1:].reset_index(drop=True) / df.iloc[:-1].reset_index(drop=True) 

,high,low,open,volumefrom,volumeto,close
0,1.052878,1.039155,0.998468,1.032000,1.070720,1.037504
1,1.091078,1.078747,1.037504,1.375652,1.509775,1.097519
2,1.069489,1.063960,1.097519,1.077974,1.163263,1.079123
3,1.098049,1.092945,1.079123,0.977052,1.030084,1.054245
4,1.082047,1.058297,1.054245,1.039348,1.154528,1.110804
...,...,...,...,...,...,...
995,0.986846,0.990890,1.016966,0.982214,0.970868,0.988448
996,0.883673,0.850861,0.988448,2.254099,1.997998,0.886385
997,0.997239,1.010333,0.886385,0.793474,0.807988,1.018291
998,1.008647,1.025426,1.018291,0.629701,0.615163,0.976913


In [35]:
df2 = df.iloc[1:].reset_index(drop=True) / df.iloc[:-1].reset_index(drop=True) 
train, test, X_train, X_test, y_train, y_test = prepare_data(df2)

In [36]:
y_train[0]

1.0375042618479373

In [37]:
X_train[0]

array([[1.05287767, 1.03915507, 0.99846809, 1.03199961, 1.07072038,
        1.03750426],
       [1.09107792, 1.07874731, 1.03750426, 1.37565154, 1.50977546,
        1.0975189 ],
       [1.06948864, 1.06396017, 1.0975189 , 1.07797378, 1.16326344,
        1.07912269],
       [1.09804871, 1.09294456, 1.07912269, 0.97705232, 1.03008362,
        1.05424528],
       [1.08204676, 1.05829655, 1.05424528, 1.03934765, 1.15452821,
        1.11080405],
       [0.98019917, 1.00392132, 1.11080405, 0.92827571, 0.90227946,
        0.97204123],
       [1.02964767, 1.01953004, 0.97204123, 0.8530495 , 0.89383076,
        1.04777575],
       [1.11102135, 1.03180491, 1.04777575, 1.45752433, 1.62175294,
        1.11265558],
       [1.16188844, 1.14923086, 1.11265558, 0.96128718, 1.03215587,
        1.07375464],
       [0.88842698, 0.96897277, 1.07375464, 0.81916611, 0.77816065,
        0.94995619]])

In [40]:
y_train[y_train > 1] = 1
y_train[y_train < 1] = 0
y_train

array([1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
       1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
       0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1.,
       1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0.,
       0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0.,
       0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
       1., 0., 0., 1., 0.

In [41]:
y_test[y_test > 1] = 1
y_test[y_test < 1] = 0
y_test

array([1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1.,
       0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0.,
       1., 0., 0., 0., 1.])

In [42]:
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.5))
model.add(Dense(units=1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
history = model.fit(X_train, y_train, epochs=200, batch_size=32, shuffle=True, verbose=1)

Epoch 1/200
28/28 [==============================] - 3s 8ms/step - loss: 0.7023 - accuracy: 0.5135
Epoch 2/200
28/28 [==============================] - 0s 9ms/step - loss: 0.6996 - accuracy: 0.5045
Epoch 3/200
28/28 [==============================] - 0s 9ms/step - loss: 0.6935 - accuracy: 0.5371: 0s - loss: 0.6847 - accuracy
Epoch 4/200
28/28 [==============================] - 0s 9ms/step - loss: 0.6946 - accuracy: 0.5090
Epoch 5/200
28/28 [==============================] - 0s 9ms/step - loss: 0.6940 - accuracy: 0.5169
Epoch 6/200
28/28 [==============================] - 0s 8ms/step - loss: 0.6938 - accuracy: 0.5056
Epoch 7/200
28/28 [==============================] - 0s 8ms/step - loss: 0.6946 - accuracy: 0.5247
Epoch 8/200
28/28 [==============================] - 0s 8ms/step - loss: 0.6970 - accuracy: 0.4820
Epoch 9/200
28/28 [==============================] - 0s 8ms/step - loss: 0.6916 - accuracy: 0.5247
Epoch 10/200
28/28 [==============================] - 0s 8ms/step - loss: 0.691

28/28 [==============================] - 0s 8ms/step - loss: 0.5130 - accuracy: 0.7438
Epoch 83/200
28/28 [==============================] - 0s 9ms/step - loss: 0.4884 - accuracy: 0.7584
Epoch 84/200
28/28 [==============================] - 0s 9ms/step - loss: 0.4263 - accuracy: 0.8090
Epoch 85/200
28/28 [==============================] - 0s 9ms/step - loss: 0.5395 - accuracy: 0.7067
Epoch 86/200
28/28 [==============================] - 0s 9ms/step - loss: 0.5139 - accuracy: 0.7539
Epoch 87/200
28/28 [==============================] - 0s 9ms/step - loss: 0.4316 - accuracy: 0.8191
Epoch 88/200
28/28 [==============================] - 0s 9ms/step - loss: 0.4303 - accuracy: 0.8101
Epoch 89/200
28/28 [==============================] - 0s 9ms/step - loss: 0.3431 - accuracy: 0.8562
Epoch 90/200
28/28 [==============================] - 0s 8ms/step - loss: 0.3896 - accuracy: 0.8281
Epoch 91/200
28/28 [==============================] - 0s 9ms/step - loss: 0.3496 - accuracy: 0.8449
Epoch 92/200


28/28 [==============================] - 0s 9ms/step - loss: 0.2202 - accuracy: 0.8978
Epoch 163/200
28/28 [==============================] - 0s 9ms/step - loss: 0.2243 - accuracy: 0.9022
Epoch 164/200
28/28 [==============================] - 0s 9ms/step - loss: 0.1863 - accuracy: 0.9236
Epoch 165/200
28/28 [==============================] - 0s 9ms/step - loss: 0.1962 - accuracy: 0.9213
Epoch 166/200
28/28 [==============================] - 0s 9ms/step - loss: 0.2170 - accuracy: 0.9135
Epoch 167/200
28/28 [==============================] - 0s 9ms/step - loss: 0.2063 - accuracy: 0.9157
Epoch 168/200
28/28 [==============================] - 0s 8ms/step - loss: 0.1849 - accuracy: 0.9292
Epoch 169/200
28/28 [==============================] - 0s 8ms/step - loss: 0.3255 - accuracy: 0.8449
Epoch 170/200
28/28 [==============================] - 0s 8ms/step - loss: 0.2134 - accuracy: 0.9101
Epoch 171/200
28/28 [==============================] - 0s 9ms/step - loss: 0.1983 - accuracy: 0.9202
Epoc

In [44]:
preds = model.predict(X_test).squeeze()

In [45]:
preds

array([0.9995746 , 0.0060226 , 0.15686768, 0.99664384, 0.01425701,
       0.07262099, 0.99952877, 0.00421035, 0.04658151, 0.46374944,
       0.99957955, 0.6068281 , 0.21961036, 0.00463799, 0.94331634,
       0.00882801, 0.20963931, 0.988612  , 0.00818285, 0.6908322 ,
       0.99719876, 0.29600042, 0.00757435, 0.9349139 , 0.99956286,
       0.9995345 , 0.9993681 , 0.99929726, 0.00445297, 0.892704  ,
       0.9971825 , 0.9986181 , 0.00802034, 0.9995664 , 0.05863512,
       0.00539112, 0.50255674, 0.22176483, 0.8006341 , 0.00996   ,
       0.02507204, 0.9964885 , 0.01124403, 0.16708234, 0.953475  ,
       0.11238214, 0.06188118, 0.07130736, 0.9995786 , 0.9434706 ,
       0.99936676, 0.9830022 , 0.5720229 , 0.8648436 , 0.9828383 ,
       0.27214015, 0.9829595 , 0.99927557, 0.85498834, 0.2530871 ,
       0.925239  , 0.05929303, 0.9995688 , 0.9993639 , 0.97475266,
       0.99955934, 0.02471587, 0.99936426, 0.13300887, 0.79465675,
       0.03268912, 0.9995492 , 0.01682219, 0.97936404, 0.00805

In [46]:
preds[preds > 0.5] = 1
preds[preds <= 0.5] = 0
preds

array([1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0.,
       1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1.,
       0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0.,
       1., 0., 0., 0., 1.], dtype=float32)

In [49]:
y_test

array([1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1.,
       0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0.,
       1., 0., 0., 0., 1.])

In [50]:
accuracy_score(y_test, preds)

0.9444444444444444